# BERT 전이학습

### 데이터 준비

In [1]:
from tensorflow.keras.utils import get_file

ratings_train_path = get_file("ratings_train.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt")
ratings_test_path = get_file("ratings_test.txt", "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt")

ratings_train_path, ratings_test_path

14628807/14628807 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4893335/4893335 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


('/root/.keras/datasets/ratings_train.txt',
 '/root/.keras/datasets/ratings_test.txt')

In [2]:
import pandas as pd

ratings_train_df = pd.read_csv(ratings_train_path, sep="\t")
ratings_test_df = pd.read_csv(ratings_test_path, sep="\t")

display(ratings_train_df.head())
display(ratings_test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [3]:
# 결측치 제거
ratings_train_df = ratings_train_df.dropna(how="any")
ratings_test_df = ratings_test_df.dropna(how="any")

display(ratings_train_df.info())
display(ratings_test_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


None

In [4]:
# 데이터 샘플링 (because 학습 시간)
ratings_train_df = ratings_train_df.sample(n=15000, random_state=0)
ratings_test_df = ratings_test_df.sample(n=5000, random_state=0)

ratings_train_df['label'].value_counts(), ratings_test_df['label'].value_counts()

(label
 0    7512
 1    7488
 Name: count, dtype: int64,
 label
 0    2532
 1    2468
 Name: count, dtype: int64)

In [5]:
X_train = ratings_train_df['document'].values.tolist()
y_train = ratings_train_df['label'].values.tolist()

X_test = ratings_test_df['document'].values.tolist()
y_test = ratings_test_df['label'].values.tolist()

In [6]:
X_train[:5]

['퇴보 된 한국영화들...씁쓸하다',
 '재밌는데?',
 '최민수가 더 건방지게 된 계기ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 김정식한테 털리놈이',
 '정말 정말 잊을수 없는 재밌는 영화... 또 보고싶다...',
 '볼만한 영화임 꼭 보셈ㅇㅇ']

### 토커나이저/모델 준비
- bert 한국어 버전 사전학습 모델 klue/bert-base

In [7]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('klue/bert-base')
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
"""
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
"""

"\nfrom transformers import BertTokenizer\n\ntokenizer = BertTokenizer.from_pretrained('klue/bert-base')\n"

In [9]:
X_train = tokenizer(X_train, padding=True, truncation=True)
X_test = tokenizer(X_test, padding=True, truncation=True)

X_train[:5], X_test[:5]

([Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=142, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])],
 [Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_tokens=118, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
  Encoding(num_

In [10]:
print(X_train['input_ids'][0])      # input_ids : 단어 토큰화
print(X_train['attention_mask'][0]) # attention_mask : 실제값/패딩 구분
print(X_train['token_type_ids'][0]) # token_type_ids : 첫번째/두번째 문장 구분 (단일 문장이라 다 0)

[2, 1800, 2178, 860, 3629, 16516, 2031, 18, 18, 18, 14242, 2205, 2062, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 데이터 파이프라인 생성

In [11]:
type(X_train)

transformers.tokenization_utils_base.BatchEncoding

In [12]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((dict(X_train), y_train))
test_ds = tf.data.Dataset.from_tensor_slices((dict(X_test), y_test))

In [13]:
train_dataset = train_ds.shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE)
test_dataset = test_ds.batch(64).prefetch(tf.data.AUTOTUNE)

### 모델 로드 & 학습

In [14]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2, from_pt=True)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from transformers import create_optimizer

num_train_steps = len(train_dataset) * 5
num_warmup_steps = int(num_train_steps * 0.1)

# optimizer
optimizer, _ = create_optimizer(
    init_lr=5e-5,                      # 학습률 초기값
    num_train_steps=num_train_steps,   # 총 학습 step수
    num_warmup_steps=num_warmup_steps, # warmup step수 (총 step수 1/10)
    weight_decay_rate=0.1              # 학습률 감쇠 계수
)

# model.hf_compute_loss == 데이터셋에 맞게 이진분류/다중분류 loss 함수 알아서 지정
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

# 학습
model.fit(train_dataset, epochs=5, validation_data=test_dataset, batch_size=64)

Epoch 1/5
235/235 [==============================] - 467s 2s/step - loss: 0.4004 - accuracy: 0.8126 - val_loss: 0.3182 - val_accuracy: 0.8618
Epoch 2/5
235/235 [==============================] - 447s 2s/step - loss: 0.2102 - accuracy: 0.9159 - val_loss: 0.3090 - val_accuracy: 0.8698
Epoch 3/5
235/235 [==============================] - 446s 2s/step - loss: 0.0960 - accuracy: 0.9657 - val_loss: 0.4054 - val_accuracy: 0.8660
Epoch 4/5
235/235 [==============================] - 446s 2s/step - loss: 0.0427 - accuracy: 0.9869 - val_loss: 0.5500 - val_accuracy: 0.8736
Epoch 5/5
235/235 [==============================] - 446s 2s/step - loss: 0.0237 - accuracy: 0.9923 - val_loss: 0.5696 - val_accuracy: 0.8754


### 모델 저장

In [ ]:
model.save_pretrained('nsmc_model/bert-base')
tokenizer.save_pretrained('msmc_model/bert-base')

### 추론

In [16]:
# 텍스트 분류 라벨 값 지정
model.config.id2label = {
    0: "부정",
    1: "긍정"
}

In [20]:
from transformers import TextClassificationPipeline

sentiment_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    # return_all_scores=True
)

Device set to use 0


In [21]:
sentiment_classifier('인생 영화를 찾았습니다.')

[{'label': '긍정', 'score': 0.9793588519096375}]

# HuggingFace

### push

In [24]:
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

In [25]:
REPO_NAME = 'bert-base-nsmc'

model.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)
tokenizer.push_to_hub(REPO_NAME, use_temp_dir=True, use_auth_token=HF_TOKEN)

/usr/local/lib/python3.11/dist-packages/transformers/modeling_tf_utils.py:3190: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tf_model.h5:   0%|          | 0.00/443M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/squirreln/bert-base-nsmc/commit/8d249c926b50fa88567b76ac415c639a4db36cd9', commit_message='Upload tokenizer', commit_description='', oid='8d249c926b50fa88567b76ac415c639a4db36cd9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/squirreln/bert-base-nsmc', endpoint='https://huggingface.co', repo_type='model', repo_id='squirreln/bert-base-nsmc'), pr_revision=None, pr_num=None)

### load

In [27]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('squirreln/bert-base-nsmc')
model = AutoModel.from_pretrained('squirreln/bert-base-nsmc', from_tf=True)

tf_model.h5:   0%|          | 0.00/443M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BertModel.

All the weights of BertModel were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


### load한 모델 사용

In [28]:
from transformers import pipeline

sentiment_classifier = pipeline('text-classification', model='squirreln/bert-base-nsmc')

Some layers from the model checkpoint at squirreln/bert-base-nsmc were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at squirreln/bert-base-nsmc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Device set to use 0


In [30]:
sentiment_classifier([
    '한국 영화는 이제 한물 갔어!',
    '어쩌라고 나는 재미있던데?ㅋㅋㅋㅋㅋㅋ',
    '혜수 언니 사랑해요',
    '볼만한 영화입니다! 추천해요!!!',
    '평생 잊지 못할 영화를 만났어요... 감동...'
])

[{'label': '부정', 'score': 0.9996700286865234},
 {'label': '긍정', 'score': 0.9966608285903931},
 {'label': '긍정', 'score': 0.9577906131744385},
 {'label': '긍정', 'score': 0.9895830154418945},
 {'label': '긍정', 'score': 0.9992875456809998}]